In [1]:
import re
import csv
from urllib.parse import urlparse, parse_qs

def parse_file(file_in, file_out):
    with open(file_in, 'r', encoding='utf-8') as fin, open(file_out, 'w', newline='', encoding='utf-8') as fout:
        csv_writer = csv.writer(fout)
        # Write the header
        csv_writer.writerow([
            'Method', 'Full URL', 'URL Path', 'Query Params', 'Content-Length', 'Content-Type',
            'User-Agent', 'Host', 'Cookie', 'Accept', 'Accept-Encoding', 'Accept-Charset',
            'Accept-Language', 'Pragma', 'Cache-Control', 'Connection', 'Body', 'Timestamp',
            'IP Address', 'Status Code'
        ])
        
        lines = fin.readlines()
        i = 0
        while i < len(lines):
            line = lines[i].strip()
            if line.startswith(('GET', 'POST', 'PUT', 'DELETE')):
                # Extract Method
                method = line.split(' ')[0]
                # Extract Full URL
                full_url = line.split(' ')[1]
                # Extract URL Path
                url_path = urlparse(full_url).path
                # Extract Query Parameters (if any)
                query_params = parse_qs(urlparse(full_url).query)
                query_params_str = '&'.join([f'{k}={v[0]}' for k, v in query_params.items()])
                
                # Initialize other fields
                content_length = ''
                content_type = ''
                user_agent = ''
                host = ''
                cookie = ''
                accept = ''
                accept_encoding = ''
                accept_charset = ''
                accept_language = ''
                pragma = ''
                cache_control = ''
                connection = ''
                body = ''
                timestamp = ''
                ip_address = ''  # Assuming IP address is not present in the line
                status_code = ''  # Assuming status code is not present in the line
                
                # Loop through headers and body
                j = 1
                while i + j < len(lines) and not lines[i + j].strip() == '':
                    header_line = lines[i + j].strip()
                    if header_line.startswith('Content-Length:'):
                        content_length = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Content-Type:'):
                        content_type = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('User-Agent:'):
                        user_agent = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Host:'):
                        host = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Cookie:'):
                        cookie = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept:'):
                        accept = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept-Encoding:'):
                        accept_encoding = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept-Charset:'):
                        accept_charset = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept-Language:'):
                        accept_language = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Pragma:'):
                        pragma = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Cache-Control:'):
                        cache_control = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Connection:'):
                        connection = header_line.split(':', 1)[1].strip()
                    j += 1
                
                # The body is the line after the headers
                if i + j < len(lines):
                    body = lines[i + j].strip()
                
                # Append the extracted features
                csv_writer.writerow([
                    method, full_url, url_path, query_params_str, content_length, content_type,
                    user_agent, host, cookie, accept, accept_encoding, accept_charset,
                    accept_language, pragma, cache_control, connection, body, timestamp,
                    ip_address, status_code
                ])
                
                i += j + 1
            else:
                i += 1



In [2]:

parse_file('anomalousTrafficTest.txt', 'anomalous_parsed_data.csv')

In [3]:
parse_file('normalTrafficTraining.txt', 'normal_parsed_data.csv')

In [4]:
import re
import csv
from urllib.parse import urlparse, parse_qs

def parse_file(file_in, file_out):
    with open(file_in, 'r', encoding='utf-8') as fin, open(file_out, 'w', newline='', encoding='utf-8') as fout:
        csv_writer = csv.writer(fout)
        # Write the header
        csv_writer.writerow([
            'Method', 'Full URL', 'URL Path', 'Query Params', 'Body Params', 'Content-Length', 
            'Content-Type', 'User-Agent', 'Host', 'Cookie', 'Accept', 'Accept-Encoding', 
            'Accept-Charset', 'Accept-Language', 'Pragma', 'Cache-Control', 'Connection'
        ])
        
        lines = fin.readlines()
        i = 0
        while i < len(lines):
            line = lines[i].strip()
            if line.startswith(('GET', 'POST', 'PUT', 'DELETE')):
                # Extract Method
                method = line.split(' ')[0]
                # Extract Full URL
                full_url = line.split(' ')[1]
                # Extract URL Path
                url_path = urlparse(full_url).path
                # Extract Query Parameters (if any)
                query_params = parse_qs(urlparse(full_url).query)
                query_params_str = '; '.join([f'{k}={v[0]}' for k, v in query_params.items()])
                
                # Initialize other fields
                content_length = ''
                content_type = ''
                user_agent = ''
                host = ''
                cookie = ''
                accept = ''
                accept_encoding = ''
                accept_charset = ''
                accept_language = ''
                pragma = ''
                cache_control = ''
                connection = ''
                body_params = ''
                
                headers = {}
                body = ''
                
                # Loop through headers and body
                j = 1
                while i + j < len(lines) and not lines[i + j].strip() == '':
                    header_line = lines[i + j].strip()
                    if header_line.startswith('Content-Length:'):
                        content_length = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Content-Type:'):
                        content_type = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('User-Agent:'):
                        user_agent = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Host:'):
                        host = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Cookie:'):
                        cookie = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept:'):
                        accept = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept-Encoding:'):
                        accept_encoding = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept-Charset:'):
                        accept_charset = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept-Language:'):
                        accept_language = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Pragma:'):
                        pragma = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Cache-Control:'):
                        cache_control = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Connection:'):
                        connection = header_line.split(':', 1)[1].strip()
                    else:
                        # Collect all header fields
                        header_key, header_value = map(str.strip, header_line.split(':', 1))
                        headers[header_key] = header_value
                    
                    j += 1
                
                # The body is the line after the headers
                if i + j < len(lines):
                    body = lines[i + j].strip()
                    # Extract body parameters (if the body is URL-encoded)
                    body_params = parse_qs(body)
                    body_params_str = '; '.join([f'{k}={v[0]}' for k, v in body_params.items()])

                # Append the extracted features
                csv_writer.writerow([
                    method, full_url, url_path, query_params_str, body_params_str, content_length, 
                    content_type, user_agent, host, cookie, accept, accept_encoding, 
                    accept_charset, accept_language, pragma, cache_control, connection
                ])
                
                i += j + 1
            else:
                i += 1




In [5]:
parse_file('anomalousTrafficTest.txt', 'anomalous_parsed_data.csv')


In [6]:
parse_file('normalTrafficTraining.txt', 'normal_parsed_data.csv')

removed features such as value of cookie as it is specific to the request

In [9]:
import re
import csv
from urllib.parse import urlparse, parse_qs
from datetime import datetime

def extract_parameters(param_str):
    """
    Extracts parameters from a URL-encoded string and returns as a formatted string.
    """
    params = parse_qs(param_str)
    return '; '.join([f'{k}={v[0]}' for k, v in params.items()])

def parse_file(file_in, file_out):
    with open(file_in, 'r', encoding='utf-8') as fin, open(file_out, 'w', newline='', encoding='utf-8') as fout:
        csv_writer = csv.writer(fout)
        # Write the header
        csv_writer.writerow([
            'Method', 'Full URL', 'URL Path', 'Query Params', 'Body Params', 
            'Content-Length', 'Content-Type', 'User-Agent', 'Host','Accept', 
            'Accept-Encoding', 'Accept-Charset', 'Accept-Language', 'Pragma', 'Cache-Control', 
            'Connection', 'Body Length', 'URL Length'
        ])
        
        lines = fin.readlines()
        i = 0
        while i < len(lines):
            line = lines[i].strip()
            if line.startswith(('GET', 'POST', 'PUT', 'DELETE')):
                # Example timestamp extraction (assuming it's available in the log)
                # This part needs to be adapted to your log format if a timestamp is present.
                timestamp = 'N/A'  # Placeholder for timestamp extraction
                
                # Extract Method
                method = line.split(' ')[0]
                
                # Extract Full URL
                full_url = line.split(' ')[1]
                
                # Extract URL Path
                url_path = urlparse(full_url).path
                
                # Extract Query Parameters
                query_params_str = extract_parameters(urlparse(full_url).query)
                
                # Initialize other fields
                content_length = ''
                content_type = ''
                user_agent = ''
                host = ''
                accept = ''
                accept_encoding = ''
                accept_charset = ''
                accept_language = ''
                pragma = ''
                cache_control = ''
                connection = ''
                body_params_str = ''
                
                headers = {}
                body = ''
                
                # Loop through headers and body
                j = 1
                while i + j < len(lines) and not lines[i + j].strip() == '':
                    header_line = lines[i + j].strip()
                    if header_line.startswith('Content-Length:'):
                        content_length = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Content-Type:'):
                        content_type = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('User-Agent:'):
                        user_agent = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Host:'):
                        host = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept:'):
                        accept = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept-Encoding:'):
                        accept_encoding = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept-Charset:'):
                        accept_charset = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Accept-Language:'):
                        accept_language = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Pragma:'):
                        pragma = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Cache-Control:'):
                        cache_control = header_line.split(':', 1)[1].strip()
                    elif header_line.startswith('Connection:'):
                        connection = header_line.split(':', 1)[1].strip()
                    j += 1
                
                # The body is the line after the headers
                if i + j < len(lines):
                    body = lines[i + j].strip()
                    # Extract body parameters (if the body is URL-encoded)
                    body_params_str = extract_parameters(body)
                
                # Calculate lengths for additional features
                body_length = len(body)
                url_length = len(full_url)
                    
                # Append the extracted features to the CSV
                csv_writer.writerow([
                    method, full_url, url_path, query_params_str, body_params_str, 
                    content_length, content_type, user_agent, host, accept, 
                    accept_encoding, accept_charset, accept_language, pragma, cache_control, 
                    connection, body_length, url_length
                ])
                
                i += j + 1
            else:
                i += 1




In [10]:
parse_file('anomalousTrafficTest.txt', 'anomalous_parsed_data.csv')


In [11]:
parse_file('normalTrafficTraining.txt', 'normal_parsed_data.csv')